Generate the LIAR Json dataset for the explanations given by the LLM model.

In [1]:
!source .env
import sys
import os
# Add the parent directory to the Python path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import json
import numpy as np
import pandas as pd
import torch
from hidden import *

/bin/bash: line 1: .env: No such file or directory


In [2]:
# Download the Liar dataset
!wget -nc https://www.cs.ucsb.edu/~william/data/liar_dataset.zip -O liar_dataset.zip

# Unzip the downloaded file
!unzip -o liar_dataset.zip -d liar_dataset
!rm liar_dataset.zip

# read train.tsv, valid.tsv, test.tsv and concatenate them
train = pd.read_csv('liar_dataset/train.tsv', sep='\t', header=None)
valid = pd.read_csv('liar_dataset/valid.tsv', sep='\t', header=None)
test = pd.read_csv('liar_dataset/test.tsv', sep='\t', header=None)
data = pd.concat([train, valid, test], axis=0)
data.columns = ['id', 'label', 'statement', 'subject', 'speaker', 'job', 'state', 'party', 'barely_true', 'false', 'half_true', 'mostly_true', 'pants_on_fire', 'venue']
data.head()

--2024-07-10 15:49:11--  https://www.cs.ucsb.edu/~william/data/liar_dataset.zip
Resolving www.cs.ucsb.edu (www.cs.ucsb.edu)... 23.185.0.253, 2620:12a:8001::253, 2620:12a:8000::253
Connecting to www.cs.ucsb.edu (www.cs.ucsb.edu)|23.185.0.253|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://sites.cs.ucsb.edu/~william/data/liar_dataset.zip [following]
--2024-07-10 15:49:11--  https://sites.cs.ucsb.edu/~william/data/liar_dataset.zip
Resolving sites.cs.ucsb.edu (sites.cs.ucsb.edu)... 128.111.27.164
Connecting to sites.cs.ucsb.edu (sites.cs.ucsb.edu)|128.111.27.164|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1013571 (990K) [application/zip]
Saving to: ‘liar_dataset.zip’

liar_dataset.zip    100%[===================>] 989.82K  1.92MB/s    in 0.5s    

2024-07-10 15:49:12 (1.92 MB/s) - ‘liar_dataset.zip’ saved [1013571/1013571]

Archive:  liar_dataset.zip
  inflating: liar_dataset/README     
  inflating: liar_data

,id,label,statement,subject,speaker,job,state,party,barely_true,false,half_true,mostly_true,pants_on_fire,venue
0,2635.json,false,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer
1,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.
2,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver
3,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release
4,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN


In [3]:
from tqdm import tqdm

# Load the model and tokenizer
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
model, tokenizer = load_model(model_id)

# Define the prompt template
prompt_template = \
"""Given a statement, predict if is true or fake. Give your classification label as True or False followed by a period.
After that, provide an explanation for your classification in one sentence followed by a period. Finally, give a certainty value from 0 to 100 representing the percentage likelihood of your statement being correct, followed by a period. For instance, \"True. The statement is true because it is backed by scientific evidence. 75.\"
Here is the statement:
"""

# Prepare statements for batch processing
statements = [prompt_template + row['statement'] for index, row in data.iterrows()]

# Process statements in batches
full_responses = batch_greedy_generation(model, tokenizer, statements)


# Extract substrings after "assistant" from each response
substrings_after_assistant = []
internal_certainty_scores = []

for res in full_responses:
    assistant_index = res.find("assistant")
    if assistant_index != -1:
        substring_after_assistant = res[assistant_index + len("assistant"):].strip()
        # Use regular expression to find the first occurrence of a numerical value in the substring
        match = re.search(r'\d+', substring_after_assistant)
        if match:
            # Convert the found numerical value to float and add to the list
            internal_certainty_scores.append(float(match.group()))
        else:
            # If no numerical value is found, append a default value or NaN
            internal_certainty_scores.append(float('nan'))
    else:
        substring_after_assistant = "The word 'assistant' was not found in the response."
        internal_certainty_scores.append(float('nan'))  # Append NaN if "assistant" not found
    substrings_after_assistant.append(substring_after_assistant)

# Add the results as new columns to the DataFrame
data['full_response'] = full_responses
data['substring_after_assistant'] = substrings_after_assistant
data['internal_certainty'] = internal_certainty_scores  # Add the internal certainty scores as a new column

# Now 'data' has two new columns: 'full_response' and 'substring_after_assistant'

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Processing batches: 100%|██████████| 200/200 [16:34<00:00,  4.97s/it]


In [4]:
data.head()

,id,label,statement,subject,speaker,job,state,party,barely_true,false,half_true,mostly_true,pants_on_fire,venue,full_response,substring_after_assistant,internal_certainty
0,2635.json,false,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer,"user\n\nGiven a statement, predict if is true ...",False.\n\nThe statement is false because Ann &...,20.0
1,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.,"user\n\nGiven a statement, predict if is true ...",False.\n\nThe statement is false because the d...,1980.0
2,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver,"user\n\nGiven a statement, predict if is true ...",False.\n\nThe statement is false because Hilla...,80.0
3,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release,"user\n\nGiven a statement, predict if is true ...",False.\n\nThe statement is false because there...,20.0
4,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN,"user\n\nGiven a statement, predict if is true ...",False.\n\nThe statement is false because the s...,20.0


In [5]:
# save data dataframe to tsv using pandas
data.to_csv('liar_dataset/data_with_responses.tsv', sep='\t', index=False)